<a href="https://colab.research.google.com/github/jacobeturpin/CXR14/blob/load-and-train/CRX14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Thorax Diseases Using the ChestX-Ray14 Dataset and Convolutional Techniques

## Introduction

Dataset provided by the National Institute of Health at: https://nihcc.app.box.com/v/ChestXray-NIHCC

*Random subset provided [here](https://www.kaggle.com/nih-chest-xrays/sample)*

## Setup

In [1]:
%tensorflow_version 2.x
!pip install keras --upgrade --quiet
!pip install efficientnet --quiet

TensorFlow 2.x selected.
     |████████████████████████████████| 378kB 2.7MB/s 


In [2]:
import gzip
import os
from urllib.request import urlretrieve

import pandas as pd

import keras
from keras.applications import DenseNet121, ResNet50V2
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

import efficientnet.keras as efn

Using TensorFlow backend.


In [0]:
DATA_PATH = './drive/My Drive/CRX14/sample/'
BATCH_SIZE = 16

CLASSES = [
  'Hernia',
  'Pneumonia',
  'Fibrosis',
  'Edema',
  'Emphysema',
  'Cardiomegaly',
  'Pleural_Thickening',
  'Consolidation',
  'Pneumothorax',
  'Mass',
  'Nodule',
  'Atelectasis',
  'Effusion',
  'Infiltration',
  'No Finding'
]

## Data Loading

### Full Dataset

In [0]:
def batch_download(first_n=None):

  # URLs for the zip files
  links = [
    'https://nihcc.box.com/shared/static/vfk49d74nhbxq3nqjg0900w5nvkorp5c.gz',
    'https://nihcc.box.com/shared/static/i28rlmbvmfjbl8p2n3ril0pptcmcu9d1.gz',
    'https://nihcc.box.com/shared/static/f1t00wrtdk94satdfb9olcolqx20z2jp.gz',
    'https://nihcc.box.com/shared/static/0aowwzs5lhjrceb3qp67ahp0rd1l1etg.gz',
    'https://nihcc.box.com/shared/static/v5e3goj22zr6h8tzualxfsqlqaygfbsn.gz',
    'https://nihcc.box.com/shared/static/asi7ikud9jwnkrnkj99jnpfkjdes7l6l.gz',
    'https://nihcc.box.com/shared/static/jn1b4mw4n6lnh74ovmcjb8y48h8xj07n.gz',
    'https://nihcc.box.com/shared/static/tvpxmn7qyrgl0w8wfh9kqfjskv6nmm1j.gz',
    'https://nihcc.box.com/shared/static/upyy3ml7qdumlgk2rfcvlb9k6gvqq2pj.gz',
    'https://nihcc.box.com/shared/static/l6nilvfa9cg3s28tqv1qc1olm3gnz54p.gz',
    'https://nihcc.box.com/shared/static/hhq8fkdgvcari67vfhs7ppg2w6ni4jze.gz',
    'https://nihcc.box.com/shared/static/ioqwiy20ihqwyr8pf4c24eazhh281pbu.gz'
  ]

  first_n = first_n or len(links)

  if first_n > len(links):
    raise('Number of files requested exceeds amount available')

  for idx, link in enumerate(links[:first_n]):
      fn = 'images_%02d.tar.gz' % (idx+1)
      print('downloading', fn, '...')
      urlretrieve(link, fn)  # download the zip file
  
  print("Download complete. Please check the checksums")

In [0]:
#batch_download(1)

In [0]:
def extract_gzip(filename):
  f = gzip.open(filename, 'rb')
  file_content = f.read()
  # TODO: implement save decompressed
  f.close()

In [0]:
for fn in os.listdir():
  if '.tar.gz' in fn:
    extract_gzip(fn)

### Sample Dataset

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [9]:
df = pd.read_csv(DATA_PATH + 'sample/sample_labels.csv')
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,5,13,060Y,M,AP,3056,2544,0.139,0.139
1,00000013_026.png,Cardiomegaly|Emphysema,26,13,057Y,M,AP,2500,2048,0.168,0.168
2,00000017_001.png,No Finding,1,17,077Y,M,AP,2500,2048,0.168,0.168
3,00000030_001.png,Atelectasis,1,30,079Y,M,PA,2992,2991,0.143,0.143
4,00000032_001.png,Cardiomegaly|Edema|Effusion,1,32,055Y,F,AP,2500,2048,0.168,0.168


In [10]:
df['Finding Labels'] = df['Finding Labels'].apply(lambda s: s.split('|'))
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y
0,00000013_005.png,"[Emphysema, Infiltration, Pleural_Thickening, ...",5,13,060Y,M,AP,3056,2544,0.139,0.139
1,00000013_026.png,"[Cardiomegaly, Emphysema]",26,13,057Y,M,AP,2500,2048,0.168,0.168
2,00000017_001.png,[No Finding],1,17,077Y,M,AP,2500,2048,0.168,0.168
3,00000030_001.png,[Atelectasis],1,30,079Y,M,PA,2992,2991,0.143,0.143
4,00000032_001.png,"[Cardiomegaly, Edema, Effusion]",1,32,055Y,F,AP,2500,2048,0.168,0.168


In [11]:
# https://stackoverflow.com/questions/45312377/how-to-one-hot-encode-from-a-pandas-column-containing-a-list

mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('Finding Labels')),
                          columns=mlb.classes_,
                          index=df.index))
df.head()

,Image Index,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
0,00000013_005.png,5,13,060Y,M,AP,3056,2544,0.139,0.139,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1
1,00000013_026.png,26,13,057Y,M,AP,2500,2048,0.168,0.168,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
2,00000017_001.png,1,17,077Y,M,AP,2500,2048,0.168,0.168,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,00000030_001.png,1,30,079Y,M,PA,2992,2991,0.143,0.143,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,00000032_001.png,1,32,055Y,F,AP,2500,2048,0.168,0.168,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0


In [12]:
df = df[['Image Index'] + CLASSES]
df.head()

,Image Index,Hernia,Pneumonia,Fibrosis,Edema,Emphysema,Cardiomegaly,Pleural_Thickening,Consolidation,Pneumothorax,Mass,Nodule,Atelectasis,Effusion,Infiltration,No Finding
0,00000013_005.png,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0
1,00000013_026.png,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
2,00000017_001.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,00000030_001.png,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,00000032_001.png,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0


## Data Preparation

The ChestX-ray14 dataset is too large to fit entirely in memory when training; therefore, it's incrementally loaded via generator to reduce memory overhead. This is achieved using the Keras [Image Proprocessing](https://keras.io/preprocessing/image/) submodule.

In [13]:
# https://datascience.stackexchange.com/a/17445/91316

train_df, test_df = train_test_split(df, test_size=0.2)

print('Training/Validation Samples:  {}'.format(len(train_df)))
print('Test Samples:  {}'.format(len(test_df)))

Training/Validation Samples:  4484
Test Samples:  1122


In [0]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.25
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [15]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=DATA_PATH + 'sample/images/',
    x_col='Image Index',
    y_col=CLASSES,
    subset='training',
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='raw',
    #classes=CLASSES,
    target_size=(224, 224)
)

Found 3363 validated image filenames.


In [16]:
valid_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=DATA_PATH + 'sample/images/',
    x_col='Image Index',
    y_col=CLASSES,
    subset='validation',
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='raw',
    #classes=[],
    target_size=(224, 224)
)

Found 1121 validated image filenames.


In [17]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=DATA_PATH + 'sample/images/',
    x_col='Image Index',
    y_col=CLASSES,
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='raw',
    #classes=[],
    target_size=(224, 224)
)

Found 1122 validated image filenames.


## Modeling

Three models will be implemented and their results compared:

1.   ResNet
2.   DenseNet
3.   EfficientNet


### ResNet

A pre-built ResNet model from the Keras library is used. Documentation on the model can be found [here](https://keras.io/applications/). Pre-trained weights from the ImageNet dataset are used.

In [18]:
resnet_base = ResNet50V2(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
)
output = Dense(15, activation='sigmoid')(resnet_base.output)

resnet = Model(input=resnet_base.input, outputs=output)

94674944/94668760 [==============================] - 2s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  if __name__ == '__main__':


In [19]:
resnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
____________________________________________________________________________________________

In [0]:
resnet.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [21]:
resnet.fit_generator(
    generator=train_generator,
    epochs=5,
    shuffle=True,
    validation_data=valid_generator
)

Epoch 1/5
177/211 [========================>.....] - ETA: 3:17 - loss: 0.2230 - accuracy: 0.9230

KeyboardInterrupt: ignored

### DenseNet

In [22]:
densenet_base = DenseNet121(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
)
output = Dense(15, activation='sigmoid')(densenet_base.output)

densenet = Model(input=densenet_base.input, outputs=output)

29089792/29084464 [==============================] - 1s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  if __name__ == '__main__':


In [23]:
densenet.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [0]:
densenet.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [25]:
densenet.fit_generator(
    generator=train_generator,
    epochs=5,
    shuffle=True,
    validation_data=valid_generator
)

Epoch 1/5
211/211 [==============================] - 639s 3s/step - loss: 0.2234 - accuracy: 0.9211 - val_loss: 0.1902 - val_accuracy: 0.9270
Epoch 2/5
146/211 [===================>..........] - ETA: 23s - loss: 0.2113 - accuracy: 0.9277

KeyboardInterrupt: ignored

### EfficientNet

EfficientNet is a lightweight CNN architecture that is designed to require significantly less compute than other state of the art architectures on common transfer learning datasets.

Pre-built EfficientNet models built in Keras are used from the efficientnet library available on [GitHub](https://github.com/qubvel/efficientnet) and installable via PyPI.

In [26]:
efficientnet_base = efn.EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
)
output = Dense(15, activation='sigmoid')(efficientnet_base.output)

efficientnet = Model(input=efficientnet_base.input, outputs=output)

16809984/16804768 [==============================] - 1s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  if __name__ == '__main__':


In [27]:
efficientnet.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 112, 112, 32) 0           stem_bn[0][0]                    
____________________________________________________________________________________________

In [0]:
efficientnet.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [29]:
efficientnet.fit_generator(
    generator=train_generator,
    epochs=5,
    shuffle=True,
    validation_data=valid_generator
)

Epoch 1/5
211/211 [==============================] - 99s 468ms/step - loss: 0.2248 - accuracy: 0.9213 - val_loss: 0.0974 - val_accuracy: 0.9236
Epoch 2/5
211/211 [==============================] - 80s 380ms/step - loss: 0.1978 - accuracy: 0.9310 - val_loss: 0.3426 - val_accuracy: 0.9302
Epoch 3/5
211/211 [==============================] - 81s 385ms/step - loss: 0.1859 - accuracy: 0.9336 - val_loss: 0.0378 - val_accuracy: 0.9260
Epoch 4/5
211/211 [==============================] - 81s 384ms/step - loss: 0.1721 - accuracy: 0.9367 - val_loss: 8.3838e-04 - val_accuracy: 0.9266
Epoch 5/5
211/211 [==============================] - 81s 383ms/step - loss: 0.1524 - accuracy: 0.9428 - val_loss: 0.0296 - val_accuracy: 0.9230


## Results

## Conclusions